In [1]:
import geopandas as gpd

gdf = gpd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\NLSDB\2025_03-03_NLSDB.gdb\Case_03032025_1240.parquet")
gdf.shape

KeyboardInterrupt: 

In [6]:
import pandas as pd

df = pd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")

In [5]:
df = pd.read_csv(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.csv", sep='|')

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (3397559015.py, line 1)

In [1]:
import os
import sys

# Set Hadoop environment variables
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['PATH'] = os.environ['PATH'] + r';C:\hadoop\bin'

# Create SparkSession with datetime rebase configuration
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("LocalSpark") \
    .config("spark.driver.memory", "16g") \
    .config("spark.sql.parquet.datetimeRebaseModeInRead", "LEGACY") \
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY") \
    .config("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY") \
    .getOrCreate()

In [2]:
gdf = spark.read.parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\NLSDB\2025_03-03_NLSDB.gdb\Case_03032025_1240.parquet")
gdf = gdf.toDF(*[col + "_nlsdb" for col in gdf.columns])

In [3]:
df = spark.read.parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")


In [4]:
df = df.toDF(
    'ID','OWNERID','ISDELETED','NAME','RECORDTYPEID','CREATEDDATE','CREATEDBYID',
    'LASTMODIFIEDDATE','LASTMODIFIEDBYID','SYSTEMMODSTAMP','LASTACTIVITYDATE',
    'LASTVIEWEDDATE','LASTREFERENCEDDATE','ACCOUNT','ACRES','BLM_ADMIN_STATE__C',
    'BLM_PRODUCT','CASE_GROUP','CASE_ID','CASE_NAME__C','CASE_STATUS','CASE_SUFFIX',
    'CASE_TYPE__C','CLAIM_IN_A_LOT','DATE_FILED','DATE_OF_LOCATION',
    'DISCOVERY_OF_VALUABLE_MINING','DISPOSITION_DATE','HEIGHT','INDEPENDENT_CASE',
    'LEGACY_ID','LEGACY_LEAD_FILE_NUMBER__C','LEGACY_SYSTEM_CODE','LNGTH','NLSDB_CASE_OID',
    'NEXT_PAYMENT_DUE_DATE','PL_359','RADIUS','RELATED_CASE','SERIAL_NUMBER__C','WIDTH',
    'REQUIRES_VOIDED_CLAIM_NOTIF','SRHA','DISP_ACT_CLSD_DT','CLSD_VOID_CLAIM_CUT_OFF_DT_YR',
    'GENERAL_REMARKS_LEGACY','LEGACY_SERIAL_NUMBER','NOITL_SERIAL_NUMBER','SYNCHRONIZATION_STATUS',
    'CASE_NAME_FORMULA','SERIAL_NUMBER_PART','META_LOAD_DT','CURRENT_MAINTENANCE_FEE_DUE',
    'LEAD_FILE_NUMBER','COMMODITY','LEASE_ISSUED_DATE','ACTION_669_FLAG','EFFECTIVE_DATE',
    'EXPIRATION_DATE','AGREEMENT_FED_PERCENT','AGREEMENT_FEE_PERCENT','AGREEMENT_INDIAN_PERCENT',
    'AGREEMENT_NONFED_PERCENT','AGREEMENT_STATE_PERCENT','NFLSS_RENTAL_AMOUNT','PARTICIPATING_FED_PERCENT',
    'PARTICIPATING_FEE_PERCENT','PARTICIPATING_INDIAN_PERCENT','PARTICIPATING_NONFED_PERCENT',
    'PARTICIPATING_STATE_PERCENT','RENTAL_RATE','TOTAL_AMOUNT','APA_APPROVED','AUTHORIZATION_LENGTH',
    'AUTHORIZATION','BILLABLE_ACRES','CASE_CATEGORY','DATE_RECEIVED','GENERATION_START_DATE',
    'GENERATION_TYPE','PRE_FLPMA','PROPOSED_ACRES','PROPOSED_LENGTH','PROPOSED_WIDTH','HOLDER_ACCOUNT',
    'HOLDER_INTEREST_PERCENT','LENGTH_MILES','IS_EXPIRED','DIAMETER_OF_PIPE_INCHES','FREQUENCY_OF_SIGNAL',
    'GEOGRAPHIC_NAME','POWERLINE_VOLTAGE_KV','PROJECT_NAME','STATUS_REASON','COMM_SITE_NAME','DATE_REPORTED',
    'DOCUMENT_CATEGORY','DOCUMENT_NUMBER','FEDERAL_ACRES_INPUT','FEE_ACRES_INPUT','US_RIGHTS_EXCEPTION',
    'US_RIGHTS','INDIAN_ACRES_INPUT','NON_FEDERAL_ACRES_INPUT','STATE_ACRES_INPUT','FEDERAL_INPUT_PERCENTAGE',
    'FEE_INPUT_PERCENTAGE','INDIAN_INPUT_PERCENTAGE','NON_FEDERAL_INPUT_PERCENTAGE','STATE_INPUT_PERCENTAGE','SYSTEM_ID',
    ##4 new attributes atarting with March export (04-08-2024)
    'PRODUCTION_STATUS','BLM_OFFICE','BLM_OFFICE_DESCR','COST_CENTER_CODE',
    ## new ones post r4 below
    'FORMATION_NAME','SALE_DATE','LAST_CASE_DISPOSITION_ACTION',
    ## 2 new  attributes added since december snapshot and defined by Natalie on 1/23 via email
    'AGREEMENT_ACRES', 'PARTICIPATING_AREA_ACRES'
    )
df = df.toDF(*[col + "_sf" for col in df.columns])

In [5]:
joined_df = gdf.join(df, gdf["SF_ID_nlsdb"] == df["ID_sf"], "left")


In [6]:
joined_df = joined_df.filter(joined_df["CSE_NR_nlsdb"] != joined_df["SERIAL_NUMBER__C_sf"])

In [7]:
joined_df.count()

2

In [7]:
joined_df.write.mode("overwrite").parquet("nlsdb_joind_blm_case.parquet")

In [8]:
joined_df = spark.read.parquet("nlsdb_joind_blm_case.parquet")
import geopandas as gpd
from shapely import wkb
import pandas as pd

# Convert the Spark DataFrame sample to pandas
pdf = joined_df.limit(50).toPandas()
# pdf[['Shape_nlsdb']]
# Convert the binary array back to bytes, then to shapely geometry
pdf['geometry'] = pdf['Shape_nlsdb'].apply(lambda x: wkb.loads(bytes(x)) if x is not None else None)

# gpd.GeoDataFrame(pdf, geometry='geometry')[44:45].plot()

# # Convert the WKB geometry column back to shapely geometry
# # The geometry column might be named 'SHAPE_nlsdb' or 'geometry_nlsdb' - adjust if needed
# pdf['geometry'] = pdf['Shape_nlsdb'].apply(lambda x: wkb.loads(x) if x else None)

# # Create a GeoDataFrame
# gdf = gpd.GeoDataFrame(pdf, geometry='geometry')

# # Create a simple plot
# ax = gdf.plot(figsize=(10, 10))
# ax.set_title('First 5 Records')

In [13]:
joined_df.limit(5).toPandas()[['CSE_NR_nlsdb','SERIAL_NUMBER__C_sf']]

,CSE_NR_nlsdb,SERIAL_NUMBER__C_sf
0,WYWY106718415,WYWY106344735
1,NDMT105823539,NDMT105485224


In [46]:
gdf.limit(5).filter(gdf['SF_ID_nlsdb']=='a023d00000FghyTAAR').toPandas()

,OBJECTID_nlsdb,CSE_NR_nlsdb,LEG_CSE_NR_nlsdb,REC_TYPE_CSE_GRP_nlsdb,BLM_PROD_nlsdb,CSE_TYPE_NR_nlsdb,CSE_LND_STATUS_nlsdb,CSE_LND_STATUS_DT_nlsdb,CSE_LND_NR_nlsdb,US_RIGHTS_nlsdb,...,SF_ID_nlsdb,SF_CL_ID_nlsdb,ID_nlsdb,AGG_CLS_ID_nlsdb,QLTY_CK_nlsdb,Created_nlsdb,Modified_nlsdb,Shape_Length_nlsdb,Shape_Area_nlsdb,Shape_nlsdb


In [56]:
gdf.select('SF_ID_nlsdb').filter(gdf['SF_ID_nlsdb']).limit(5).toPandas()#[['SF_ID_nlsdb']]

,SF_ID_nlsdb
